In [1]:
pwd

'C:\\Users\\clubc\\Desktop\\python_basic'

In [2]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('./tmdb_5000_movies.csv')
print(movies.shape)
movies.head(3)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [3]:
# 추천 시스템에 사용할 주요 칼럼만 추출

movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [4]:
pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:2]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is..."


In [5]:
from ast import literal_eval # 문자열을 문자열이 의미하는 list[dict1, dict2] 객체로 만들 수 있다.
movies_df['genres'] = movies_df['genres'].apply(literal_eval) 
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [6]:
# genres 컬럼에서 'name'키에 해당하는 값을 추출하기 위해 lambda 식 사용
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x]) 
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


In [14]:
# genres를 문자열로 변경한 뒤 이것을 CounterVectorizer로 벡터화

from sklearn.feature_extraction.text import CountVectorizer

# CounterVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df = 0, ngram_range = (1, 2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(movies_df['genres_literal'])
print(genre_mat.shape)

0        Action Adventure Fantasy Science Fiction
1                        Adventure Fantasy Action
2                          Action Adventure Crime
3                     Action Crime Drama Thriller
4                Action Adventure Science Fiction
5                        Fantasy Action Adventure
6                                Animation Family
7                Action Adventure Science Fiction
8                        Adventure Fantasy Family
9                        Action Adventure Fantasy
10       Adventure Fantasy Action Science Fiction
11                Adventure Action Thriller Crime
12                       Adventure Fantasy Action
13                       Action Adventure Western
14       Action Adventure Fantasy Science Fiction
15                       Adventure Family Fantasy
16               Science Fiction Action Adventure
17                       Adventure Action Fantasy
18                  Action Comedy Science Fiction
19                       Action Adventure Fantasy


In [8]:
# 코사인 유사도를 행렬형태로 변환

from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


In [15]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:4])

[[   0 3494  813 ... 3038 3037 2401]
 [ 262    1  129 ... 3069 3067 2401]
 [   2 1740 1542 ... 3000 2999 2401]
 [2195 1850 3316 ...  887 2544 4802]]


In [34]:
# 장르별로 유사한 영화 찾는 함수

def find_sim_movie_v1(df, sorted_ind, title_name, top_n = 10):
    # 인자로 입력된 movies_df에서 'title' 컬럼이 입력된 title_name 값인 DataFrame을 추출한다.
    title_movie = df[df['title'] == title_name]
    
    # title_name을 가진 DataFrame의 index 객체를 ndarry로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개 만큼 index를 추출한다.
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :top_n]
    
    # 추출된 top_n index 출력. top_n index는 2차원 데이터
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

In [46]:
similar_movies = find_sim_movie_v1(movies_df, genre_sim_sorted_ind, 'Avatar', 10)
similar_movies[['title', 'vote_average']]

[[   0 3494  813  870   46   14 1296 1652  419  420]]


,title,vote_average
0,Avatar,7.2
3494,Beastmaster 2: Through the Portal of Time,4.6
813,Superman,6.9
870,Superman II,6.5
46,X-Men: Days of Future Past,7.5
14,Man of Steel,6.5
1296,Superman III,5.3
1652,Dragonball Evolution,2.9
419,Jumper,5.9
420,Hellboy II: The Golden Army,6.5


In [36]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending = False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [37]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:', round(C, 3), 'm:', round(m, 3))

C: 6.092 m: 370.2


In [48]:
# v = 개별 영화에 평점을 투표한 횟수
# m = 평점을 부여하기 위한 최소 투표 횟수
# R = 개별 영화에 대한 평균 평점
# C = 전체 영화에 대한 평균 평점

percentile = 0.6
m = movies['vote_count'].quantile(percentile)
C = movies['vote_average'].mean()

# 가중 평점을 만들기 위한 함수
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v + m) * R) + ( (m / (m + v)) * C))

In [49]:
movies_df['weighted_vote'] = movies.apply(weighted_vote_average, axis = 1)
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending = False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [50]:
# 장르별로 유사한 영화 찾는 함수
# 후보군 늘리고 weighted_vote 컬럼의 값이 높은 순으로 추출

def find_sim_movie_v2(df, sorted_ind, title_name, top_n = 10):
    # 인자로 입력된 movies_df에서 'title' 컬럼이 입력된 title_name 값인 DataFrame을 추출한다.
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # 후보군 2배로 늘림
    similar_indexes = sorted_ind[title_index, :(top_n * 2)] 
    similar_indexes = similar_indexes.reshape(-1)
    
    # 기존 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n 의 2배만큼의 후보군에서 weighted_vote 가 높은 순으로 top_n 만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending = False)[:top_n]

In [51]:
similar_movies = find_sim_movie_v2(movies_df, genre_sim_sorted_ind, 'Avatar', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
46,X-Men: Days of Future Past,7.5,7.418594
813,Superman,6.9,6.685190
14,Man of Steel,6.5,6.477564
420,Hellboy II: The Golden Army,6.5,6.420421
870,Superman II,6.5,6.348901
232,The Wolverine,6.3,6.282606
3208,Star Wars: Clone Wars: Volume 1,8.0,6.221858
1191,Small Soldiers,6.2,6.154700
1932,Sheena,5.0,6.030907
3494,Beastmaster 2: Through the Portal of Time,4.6,6.026658
